# CS 559: Final Project
Yashvi, Annanya Chaturvedi, Zheng Ke Xu

## 1. Feature Selection
Analyze 95 column data and select the most relevant features for determining bankruptcy.
- Section 3.1: Training data process for Clustering

In [301]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('./train_data.csv').drop(columns=["Index", " Net Income Flag"])

def preprocess_data(df):
    """Performs standardization and feature selection on training data set"""
    df_numpy = df.to_numpy()
    scalar = StandardScaler()
    X_scaled = scalar.fit_transform(df_numpy)

    df_scaled = pd.DataFrame(
        X_scaled,
        columns=df.columns,
        index=df.index
    )

    corr_matrix = df_scaled.corr()

    bankrupt_feature = corr_matrix["Bankrupt?"]
    abs_corr = bankrupt_feature[abs(bankrupt_feature) > 0.1].drop("Bankrupt?")
    features = df_scaled[abs_corr.index.tolist()]

    print(features)

    return features, corr_matrix

joblib.dump(preprocess_data, 'preprocess_data.joblib')
print("Custom preprocessor for training data 'preprocess_data' saved to 'preprocess_data.joblib'. Import to transform data.")

Custom preprocessor for training data 'preprocess_data' saved to 'preprocess_data.joblib'. Import to transform data.


- Running preprocess_data on training data set. This performs standardization and feature selection automatically.

In [302]:
processed_features, corr_matrix = preprocess_data(df=df)
selected_features = processed_features

print(selected_features)

      ROA(C) before interest and depreciation before interest  \
0                                             -0.904871         
1                                              0.404409         
2                                              1.081097         
3                                             -0.362077         
4                                              0.081298         
...                                                 ...         
5802                                           0.033994         
5803                                           0.795670         
5804                                          -0.226579         
5805                                           0.135016         
5806                                           0.532691         

      ROA(A) before interest and % after tax  \
0                                  -0.835696   
1                                   0.213147   
2                                   0.933137   
3                          

- The main idea here is to select for features that have a correlation value to Bankruptcy above or below 0.1 (weak correlation)
- See below for how the correlation matrix looks like.

In [303]:
corr_matrix.style.background_gradient(cmap='coolwarm')

## 2. Clustering Subgroups
Determining subgroups from selected features using K-means unsupervised learning model.
- Section 3.2: Company Characterization
- Start by training KMeans on selected features

In [304]:
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters=6, n_init=10, random_state=100, init='k-means++')
k_means.fit(selected_features.to_numpy())

labels = k_means.labels_
unique_labels = pd.Series(labels).value_counts()
centroids = k_means.cluster_centers_ 

print("K-means labels: \n" + str(labels) + "\n")
print("K-means unique labels: \n" + str(unique_labels) + "\n")
print("K-means centroids: \n" + str(centroids))

K-means labels: 
[1 2 2 ... 1 2 2]

K-means unique labels: 
2    2379
1    2292
3     682
0     444
4       7
5       3
Name: count, dtype: int64

K-means centroids: 
[[ 1.68016643e+00  1.60494023e+00  1.60925902e+00  1.09862957e-01
   2.05128544e+00  2.04672919e+00  2.04567996e+00  2.15437349e+00
   2.01714695e+00  2.12007847e+00 -4.38994331e-01  4.38994331e-01
  -2.02573750e-01  2.00992760e+00  2.12395116e+00  7.02320531e-01
   9.35812672e-01 -2.54615636e-01  1.89551487e-01 -1.03904167e-01
   1.08894922e+00 -1.37730746e-01 -1.03904167e-01 -1.37955676e-01
   8.09255720e-01 -3.09588200e-01 -3.47404167e-02  1.35983855e+00
   2.64952414e-01 -1.29013280e-01]
 [-1.58166089e-01 -9.83395527e-02 -1.33079552e-01  1.35436709e-01
  -2.07511549e-01 -2.05508169e-01 -2.05104264e-01 -1.59634529e-01
  -1.34862606e-01 -1.54171638e-01  6.53137558e-01 -6.53137558e-01
   1.52949971e-01 -1.35059618e-01 -1.59245945e-01 -4.24878501e-01
  -4.25929440e-01  5.39523158e-01 -4.41222910e-02  1.38997180e-01
  -2.6

Once labels are available, group the selected features by their labels. This creates subgroups for further analysis. Any subgroups without any bankrupt entries are set as h=0.

In [329]:
selected_features['Cluster'] = labels
grouped_features = selected_features.groupby('Cluster')

#Save each group as its own array - makes it easier to distribute among the group
subgroups = {}
for label, group in grouped_features:
    subgroups[label] = group.to_numpy()

#Since each label changes between runtimes, we should sort the subgroups to keep it consistent
sorted_subgroups = sorted(
    subgroups.items(),
    key=lambda item: len(item[1]),
    reverse=True
)

sorted_subgroups = [item[1] for item in sorted_subgroups]
for i in sorted_subgroups:
    print(len(i))

2379
2292
682
444
7
3


## 3. Training Models
Create individual models to process the subgroups.
- Section 3.3: Building Training Models